In [1]:
## Ali Akbari
## Data 311 Assignment #4
## Binary Search Tree & Heap Array Implementation

# helper function
def print_tree(n, indent = 0):
    if n is None:
        print( " " * indent, "X")
        return
    print_tree(n.right, indent + 4)
    print( " " * indent, n.val)
    print_tree(n.left, indent + 4)
# helper function
def inorder(n):
    if not n: return
    yield from inorder(n.left)
    yield n.val
    yield from inorder(n.right)

class BST:
    class Node:
        def __init__(self, val):
            self.val = val
            self.left = None
            self.right = None

    def __init__(self):
        self.root = None

    def insert(self, val):
        n = self.Node(val)
        if self.root is None:
            self.root = n
            return
        r = self.root
        while r is not None:
            if val < r.val:
                if r.left is None:
                    r.left = n
                    return
                else:
                    r = r.left
            else:
                if r.right is None:
                    r.right = n
                    return
                else:
                    r = r.right

    def __iter__(self):
        yield from inorder(self.root)

    def print(self):
        print("------- Tree -----------")
        if self.root is None:
            print("Empty tree")
        else:
            print_tree(self.root)
        print("------------------------")
        
    def minValueNode(self, node):
        current = node

        # loop down to find the leftmost leaf
        while(current.left is not None):
            current = current.left

        return current
        
    def remove(self, value):
        #Checking if tree is empty, if there is a root
        #Return nothing
        if self.root is None:
            return
        
        #If root node value is the value we are looking for 
        elif self.root.val == value:
            #Checking if there is no child
            #If there is no child, remove root node
            if self.root.left is None and self.root.right is None:
                self.root = None
            #Checking if there is a left child and no right child
            #Make left child the root node
            elif self.root.left and self.root.right is None:
                self.root = self.root.left
            #Checking if there is a right child and no left child
            #Make right child the root node
            elif self.root.left is None and self.root.right:
                self.root = self.root.right
            #Checking if there are both left & right children
            #Find successor to replace root node
            #Check and find the leftmost closest value to root node by searching the right subtree
            elif self.root.left and self.root.right:
                remNode = self.root.right
                remNodeParent = self.root
            #Loop to find leftmost
                while remNode.left:
                    remNodeParent = remNode
                    remNode = remNode.left
            #Changing root value and fixing the pointers to right subtree
                self.root.val = remNode.val
                if remNode.right:
                    if remNodeParent.val > remNode.val:
                        remNodeParent.left = remNode.right
                    else:                                 #remNodeParent.val < remNode.val:
                        remNodeParent.right = remNode.right
                else:
                    if remNode.val < remNodeParent.val:
                        remNodeParent.left = None
                    else:
                        remNodeParent.right = None

            return True

        #Initializing parent node and temporary node
        parentNode = None
        node = self.root

        #Finding the node with the value to remove
        while node and node.val != value:
            parentNode = node
            if value < node.val:
                node = node.left
            elif value > node.val:
                node = node.right

        #If not in the tree print message and return False
        if node is None or node.val != value:
            print("No such element in tree")
            return False

        #Checking if there is no child
        #If there is no child, remove node using parent Node
        elif node.left is None and node.right is None:
            if value < parentNode.val:
                parentNode.left = None
            else:
                parentNode.right = None
            return True

        #Checking if there is a left child and no right child
        #Use left child of the node for the parent node 
        elif node.left and node.right is None:
            if value < parentNode.val:
                parentNode.left = node.left
            else:
                parentNode.right = node.left
            return True

        #Checking if there is a right child and no left child
        #Use right child of the node for the parent node 
        elif node.left is None and node.right:
            if value < parentNode.val:
                parentNode.left = node.right
            else:
                parentNode.right = node.right
            return True

        #Checking if there are both left & right children
        #Find successor to replace root node
        #Check and find the leftmost closest value to removed node by searching the right subtree
        else:
            remNodeParent = node
            remNode = node.right
        #Loop to find leftmost
            while remNode.left:
                remNodeParent = remNode
                remNode = remNode.left
                
        #Changing root value and fixing the pointers to right subtree
            node.val = remNode.val
            if remNode.right:
                if remNodeParent.val > remNode.val:
                    remNodeParent.left = remNode.right
                elif remNodeParent.val < remNode.val:
                    remNodeParent.right = remNode.right
            else:
                if remNode.val < remNodeParent.val:
                    remNodeParent.left = None
                else:
                    remNodeParent.right = None

In [2]:
#heapq import used orginally
import heapq

#Returns the parent of a node
#heap index is passed in to calculate the parent index
def getParent(i): return (i-1) // 2

#Returns the left child a node
#heap index is passed in to calculate the left index 
def getLeft(i): return 2 * i + 1

#Returns the right child a node
#heap index is passed in to calculate the right index
def getRight(i): return 2 * i + 2

#Algorithm to swap nodes upwards of the heap until in correct position
#Heap properties are kept
def siftup(heap, index):
#loop will break/return when parent value is less than node insert value
    while index > 0:
        parentIndex = getParent(index)
        if heap[parentIndex] <= heap[index]:
            return
        else:
            heap[parentIndex], heap[index] = heap[index], heap[parentIndex]
            index = parentIndex

#Algorithm to swap nodes downwards of the heap until in correct position
#Heap properties are kept
def siftdown(heap, rootIndex, lastIndex):
    while getLeft(rootIndex) <= lastIndex: 
        childNodeIndex = getLeft(rootIndex)
        swapNodeIndex = rootIndex
        if heap[swapNodeIndex] > heap[childNodeIndex]:
            swapNodeIndex = childNodeIndex
        if childNodeIndex + 1 <= lastIndex and heap[swapNodeIndex] > heap[childNodeIndex + 1]:
            swapNodeIndex = childNodeIndex + 1
        if swapNodeIndex == rootIndex:
            return
        else:
            heap[rootIndex], heap[swapNodeIndex] = heap[swapNodeIndex], heap[rootIndex]
            rootIndex = swapNodeIndex
            

#Algorithm to push values into heap
#Heap properties are kept
def my_heappush(heap, value):
    heap.append(value)
    lastIndex = len(heap) - 1
    siftup(heap, lastIndex)
#Run time of O(logn) as it is dependent on the hieght of the tree
#Original python heapq function used 
#heapq.heappush(heap,value)

#Algorithm to pop top value out of heap
#Returns minimum values as this is min heap
#Heap properties are kept
def my_heappop(heap):
    if len(heap) <= 0:
        print('Index error: pop operation from empty heap')
        return
    minValue = heap[0]
    heap[0] = heap[len(heap) - 1]
    siftdown(heap, 0, len(heap) -1)    
    del heap[-1]
    return minValue
#Run time of O(logn) as it is dependent on the hieght of the tree 
#Original python heapq function used
#return heapq.heappop(heap)


#Heapify algorithm that uses sift down function to make heap of a random array
def my_heapify(heap):
    lenOfHeap = len(heap)
    parentNode = getParent(lenOfHeap -1)
    while parentNode >= 0:
        siftdown(heap, parentNode, lenOfHeap -1)
        parentNode = parentNode - 1 
#Run time of O(n) as it check every element 
#Original python heapq function used    
#heapq.heapify(heap)

#Sorting algorithm  for heap using heapify function to make array to heap first
#Uses siftdown algorithm 
def my_heapsort(heap):
    my_heapify(heap)
    #Runs in O(n)
    for i in range(len(heap) - 1, 0, -1):
        heap[i], heap[0] = heap[0], heap[i]
        #Runs in O(logn)
        siftdown(heap, 0, i-1)       
#Min heap gives us descending order, and we need ascending order for sort
#Heap being built again by reversing the array so we end up with sorted array
    leftElement, rightElement =  0, len(heap) -1
    while leftElement < rightElement:
        heap[leftElement], heap[rightElement] = heap[rightElement], heap[leftElement]
        rightElement -= 1
        leftElement += 1            
#Original python heapq function used 
#heap.sort()


###################### Non modified function given ##############################
# you can use the tests below to verify your implementations are correct
# but do not modify the tests below

def is_heap(heap):
    def parent(i): return (i-1) // 2
    def left(i): return 2 * i + 1
    def right(i): return 2 * i + 2
    for i in range(0, parent(len(heap)-1)):
        if left(i) < len(heap) and heap[i] > heap[left(i)]: return False
        if right(i) < len(heap) and heap[i] > heap[right(i)]: return False
    return True

#Non modified test functions given 
import random
def test_my_heappush(n):
    print("Testing my_heappush:")
    a = [random.randint(0,100) for x in range(n)]
    print(" [I] inserting:", a)
    heap = []
    for i in a:
        my_heappush(heap,i)
    print(" [I] heap:", heap)
    if sorted(heap) != sorted(a):
        print(" [E] incorrect elements in heap after inserting:")
    if not is_heap(heap):
        print(" [E] not a heap after inserting")
def test_my_heappop(n):
    print("Testing my_heappop:")
    a = [random.randint(0,100) for x in range(n)]
    heapq.heapify(a)
    print(" [I] testing pop on heap:", a)
    heapcopy = a[:]
    true_min = heapq.heappop(heapcopy)
    my_min = my_heappop(a)
    print(" [I] popped item:", my_min)
    print(" [I] resulting heap:", a)
    if true_min != my_min:
        print(" [I] my_heappop returned", my_min)
        print(" [I] but real min is", true_min)
    if sorted(heapcopy) != sorted(a):
        print(" [E] incorrect elements in heap after popping")
    if not is_heap(a):
        print(" [E] not a heap after popping")
        
def test_my_heapify(n):
    print("Testing my_heapify:")
    a = [random.randint(0,100) for x in range(n)]
    print(" [I] input array:", a)
    heapcopy = a[:]
    heapq.heapify(heapcopy)
    my_heapify(a)
    print(" [I] heap:", a)
    if sorted(heapcopy) != sorted(a):
        print(" [E] incorrect elements in heap after heapify")
    if not is_heap(a):
        print(" [E] not a heap after heapify")

def test_my_heapsort(n):
    print("Testing my_heapsort")
    a = [random.randint(0,100) for x in range(n)]
    copy = a[:]
    my_heapsort(a)
    print(" [I] input array:", copy)
    print(" [I] sorted output:", a)
    if sorted(a) != sorted(copy):
        print(" [E] incorrect elements in sorted array")
    if a != sorted(a):
        print(" [E] not sorted")
    
def test_all(n):
#Examples for insert and remove for binary search tree
#     bst = BST()
#     bst.insert(2)
#     bst.insert(3)
#     bst.insert(4)
#     bst.insert(1)
#     bst.insert(6)
#     bst.insert(12)
#     bst.insert(11)
#     bst.print()
#     bst.remove(3)
#     bst.print()
#     bst.remove(6)
#     bst.print()
#     bst.remove(8)
#     bst.print()
#     bst.remove(2)
#     bst.print()
    test_my_heappush(n)
    test_my_heappop(n)
    test_my_heapify(n)
    test_my_heapsort(n)
test_all(9)

Testing my_heappush:
 [I] inserting: [19, 51, 34, 71, 79, 70, 72, 81, 66]
 [I] heap: [19, 51, 34, 66, 79, 70, 72, 81, 71]
Testing my_heappop:
 [I] testing pop on heap: [6, 15, 28, 51, 63, 64, 40, 55, 54]
 [I] popped item: 6
 [I] resulting heap: [15, 51, 28, 54, 63, 64, 40, 55]
Testing my_heapify:
 [I] input array: [52, 64, 23, 27, 9, 93, 83, 31, 26]
 [I] heap: [9, 26, 23, 27, 64, 93, 83, 31, 52]
Testing my_heapsort
 [I] input array: [14, 80, 62, 8, 79, 86, 97, 40, 95]
 [I] sorted output: [8, 14, 40, 62, 79, 80, 86, 95, 97]
